Comprobación de PyTorch y uso de la GPU

In [ ]:
import torch

print(f"Versión de PyTorch: {torch.__version__}")
print(f"Versión de CUDA en PyTorch: {torch.version.cuda}")
print(f"¿CUDA está disponible?: {torch.cuda.is_available()}")
print(torch.cuda.get_device_name(0))

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n-seg.pt")

results = model.train(data="./datasets/basketball_court_zones_segmentation_v2/data.yaml", imgsz=640, batch=8, device=0, epochs=60, task="segment")

Código reutilizado de BCT

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO  #Necesario para detectar con YOLO

#Intersección enter dos líneas
def line_intersection(line1, line2):
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Calcular  pendientes e intersecciones
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if denom == 0:  # Líneas paralelas o coincidentes
        return None

    intersect_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
    intersect_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom

    return (intersect_x, intersect_y)

#Comprueba que el punto esté dentro de la imagen
def is_within_image(point, image_width, image_height):
    x, y = point
    return 0 <= x < image_width and 0 <= y < image_height


# Cargar el modelo YOLO entrenado
model = YOLO("./runs/segment/train/weights/best.pt")

video_path = "../clips/Clip1LF.mp4"
# Cargar el video de entrada
cap = cv2.VideoCapture(video_path)

# Obtener detalles del video
fourcc = cv2.VideoWriter_fourcc(*"avc1")  # Codec para el video de salida
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Realizar detección en el frame
    results = model(frame)
    # Dimensiones del frame original
    frame_height, frame_width = frame.shape[:2]

    # Iterar sobre los resultados
    for result in results:
        masks = result.masks.data.cpu().numpy()  # Máscaras (NumPy array)
        classes = result.boxes.cls.cpu().numpy()  # Clases detectadas
        boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes
        image = frame.copy()  # Crear copia del frame original para visualización

        # Aplicar y mostrar máscaras
        for i, cls in enumerate(classes):
            if model.names[cls] == 'paint':
                # Redimensionar la máscara al tamaño del frame original
                zone_mask = (masks[i] * 255).astype(np.uint8)  # Escalar la máscara (0-255)
                resized_zone_mask = cv2.resize(zone_mask, (frame_width, frame_height), interpolation=cv2.INTER_NEAREST)

                # Superponer la máscara en el frame original
                image[resized_zone_mask > 0] = (0, 0, 255)  # Rojo


    # Mostrar la zona en pantalla
    cv2.imshow("La zona", image)
    # Esperar por una tecla: espacio para avanzar, 'q' para salir
    key = cv2.waitKey(0) & 0xFF  # Espera indefinidamente hasta que se presione una tecla
    if key == ord('q'):
        break

# Liberar recursos
cap.release()

cv2.destroyAllWindows()

# Esperar a que se presione una tecla y cerrar ventana
cv2.waitKey(0)

cv2.destroyAllWindows()